In [10]:
import pygame
import numpy as np
import sys
from enum import Enum

WIDTH, HEIGHT = 800, 800
CELL_SIZE = 4
ROWS, COLS = HEIGHT // CELL_SIZE, WIDTH // CELL_SIZE

class AntDirection(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3

class Cell(Enum):
    WHITE = 0
    BLACK = 1
    
class Color(Enum):
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)
    WHITE = (255, 255, 255)

class LangtonAnt:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = np.zeros((rows, cols), dtype=int)
        self.ant_pos = [rows // 2, cols // 2]
        self.ant_direction = AntDirection.UP.value
        self.steps_per_frame = 1
        self.speed_multiplier = 1.0
        self.step = 0
        self.fps = 5
    
    def move_multiple(self):
        """Выполняет несколько шагов за один кадр"""
        for _ in range(int(self.steps_per_frame * self.speed_multiplier)):
            x, y = self.ant_pos
            
            if self.grid[x, y] == Cell.WHITE.value:
                self.ant_direction = (self.ant_direction + AntDirection.RIGHT.value) % 4
                self.grid[x, y] = Cell.BLACK.value
            else:
                self.ant_direction = (self.ant_direction - AntDirection.RIGHT.value) % 4
                self.grid[x, y] = Cell.WHITE.value
                
            if self.ant_direction == AntDirection.UP.value: x = (x - 1) % self.rows
            elif self.ant_direction == AntDirection.RIGHT.value: y = (y + 1) % self.cols
            elif self.ant_direction == AntDirection.DOWN.value: x = (x + 1) % self.rows
            elif self.ant_direction == AntDirection.LEFT.value: y = (y - 1) % self.cols
                
            self.ant_pos = [x, y]
            self.step += 1
    
    def increase_speed(self):
        """Увеличивает скорость"""
        if self.fps < 120:
            self.fps *= 2
        if self.speed_multiplier < 1000:
            self.speed_multiplier *= 2
    
    def decrease_speed(self):
        """Уменьшает скорость"""
        if self.fps > 4:
            self.fps /= 2
        if self.speed_multiplier > 1:
            self.speed_multiplier /= 2
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Муравей Лэнгтона - Управление: ↑/↓ скорость, Пробел: пауза")
    
    ant = LangtonAnt(ROWS, COLS)
    clock = pygame.time.Clock()
    
    running = True
    paused = False
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    paused = not paused
                elif event.key == pygame.K_UP:
                    ant.increase_speed()
                elif event.key == pygame.K_DOWN:
                    ant.decrease_speed()
                elif event.key == pygame.K_r:
                    ant = LangtonAnt(ROWS, COLS)
                elif event.key == pygame.K_ESCAPE:
                    running = False
        
        if not paused:
            ant.move_multiple()
        
        screen.fill(Color.WHITE.value)
        
        for i in range(ant.rows):
            for j in range(ant.cols):
                if ant.grid[i, j] == Cell.BLACK.value:
                    pygame.draw.rect(screen, Color.BLACK.value, 
                                   (j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        
        ant_x, ant_y = ant.ant_pos
        pygame.draw.rect(screen, Color.RED.value, 
                       (ant_y * CELL_SIZE, ant_x * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        
        font = pygame.font.SysFont('Arial', 16)
        info_text = [
            f'Скорость: x{ant.speed_multiplier:.1f}',
            f'Шагов/кадр: {int(ant.fps)}',
            f'Общее шагов: {ant.step}',
            'Управление: ↑/↓ скорость, Пробел: пауза, R: сброс'
        ]
        
        for i, text in enumerate(info_text):
            rendered = font.render(text, True, Color.BLACK.value)
            screen.blit(rendered, (10, 10 + i * 20))
        
        pygame.display.flip()
        clock.tick(ant.fps)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

SystemExit: 